# AI final project - Connect Four

Pouya Mohseni - 610398164

In [1]:
import numpy as np
import random
import os

#define a class for connect four 
class cnctf():
    column: int #number of columns in the game
    row: int #number of rows in the game
    game: int #the game board
    
    def __init__(self,row,column,playing_type=0):
        if playing_type==1: returnself.game_train() #for nueral net training
            
        os.system('cls')
        print("**Welcome to Connect Four!**")
        print("Choose the game board size: ")
        self.row = int(input("Number of rows: "))
        self.column = int(input("Number of columns: "))
        
        self.game = [[0 for i in range(self.column)] for j in range(self.row)]
                          
        player1 = []
        player1.append(input("What is the name of the player 1: "))
        player1.append(input("What is the player 1 piece (preferably X): "))
        player1.append(input("Type AI or PLAYER regarding to player 1 agent: "))
        if player1[-1]=="AI":player1.append(int(input("Determine player 1 difficulty (2-5): ")))
        if player1[-2]=="AI":player1.append(input("Choose search alogorithm for player 1 (MinMax - AlphaBeta - A*): "))
        
        player2 = []
        player2.append(input("What is the name of the player 2: "))
        player2.append(input("What is the player 2 piece (preferably O): "))
        player2.append(input("Type AI or PLAYER regarding to player 2 agent: "))
        if player2[-1]=="AI":player2.append(int(input("Determine player 2 difficulty (2-5): ")))
        if player2[-2]=="AI":player2.append(input("Choose a search alogorithm for player 2 (MinMax - AlphaBeta - A*): "))
        
            
        self.game_turner(player1,player2)
        game_go = int(input("Do you want to continue? (0: No - 1: Yes): "))
        if game_go == 1: self.__init__()
        
    def game_train(self):
        #define arrays to have maze and moves
        self.input_ = []
        self.label_ = []
        
        #define players
        p1=["Random","X","PLAYER"]
        p2=["AIname","O","AI",4,"AlphaBeta"]
        game_num = 0
        end_game = 0
        for game_num in range(self.row*self.column):
            if end_game: break
            if game_num%2==0:
                move = random.choice(self.possible_moves())
                if self.player_move(move,p1[:2],p2[:2]): end_game = 1
                    
            elif game_num%2==1:
                ai = AlphaBeta_AI(p2[4],self.row,self.column)
                move = ai.search(self.game)
                self.input_.append(self.game[0]+self.game[1]+self.game[2]+self.game[3]+self.game[4]+self.game[5]+self.game[6])
                self.label_.append(move)
                if self.player_move(move,p2[:2],p1[:2]): end_game = 1
                
        return [self.input_,self.label_]
    
    def game_turner(self,p1=["Name1","X","PLAYER"],p2=["AIname","O","AI",6,"A*"]):
        os.system('cls')
        game_num = 0
        end_game = 0
        for game_num in range(self.row*self.column):
            if end_game: break
            if game_num%2==0:
                #move for p1
                if p1[2]=="PLAYER":
                    #agent is controled through the shell
                    played=0
                    while not played:
                        move = int(input("Tosse a piece(0-"+str(self.column-1)+"): "))
                        if move in self.possible_moves():
                            played=1
                            if self.player_move(move,p1[:2],p2[:2]): end_game = 1
                            
                if p1[2]=="AI":
                    #agent is controled through an AI
                    if p1[4]=="MinMax":
                        ai = MinMax_AI(p1[3],self.row,self.column)
                        move = ai.search(self.game)
                        if self.player_move(move,p1[:2],p2[:2]): end_game = 1
                    if p1[4]=="AlphaBeta":
                        ai = AlphaBeta_AI(p1[3],self.row,self.column)
                        move = ai.search(self.game)
                        if self.player_move(move,p1[:2],p2[:2]): end_game = 1
                    if p1[4]=="A*":
                        ai = Astar_AI(p1[3],self.row,self.column)
                        move = ai.search(self.game)
                        if self.player_move(move,p1[:2],p2[:2]): end_game = 1
                            
            elif game_num%2==1:
                #move for p2
                if p2[2]=="PLAYER":
                    #agent is controled through the shell
                    played=0
                    while not played:
                        move = int(input("Tosse a piece(0-"+str(self.column-1)+"): "))
                        if move in self.possible_moves():
                            played=1
                            if self.player_move(move,p2[:2],p1[:2]): end_game = 1
                            
                if p2[2]=="AI":
                    #agent is controled through an AI
                    if p2[4]=="MinMax":
                        ai = MinMax_AI(p2[3],self.row,self.column)
                        move = ai.search(self.game)
                        if self.player_move(move,p2[:2],p1[:2]): end_game = 1
                    if p2[4]=="AlphaBeta":
                        ai = AlphaBeta_AI(p2[3],self.row,self.column)
                        move = ai.search(self.game)
                        if self.player_move(move,p2[:2],p1[:2]): end_game = 1
                    if p2[4]=="A*":
                        ai = Astar_AI(p2[3],self.row,self.column)
                        move = ai.search(self.game)
                        if self.player_move(move,p2[:2],p1[:2]): end_game = 1
                    
    def possible_moves(self):
        #find all possible moves in a possition
        pm = list(range(self.column))
        pm_copy = pm[:]
        for i in pm_copy:
            if self.game[0][i]!=0: pm.remove(i)
        return pm
    
    def player_move(self,move,p1=['NAME1','X'],p2=['AI','O']):
        pm = self.possible_moves()
        
        #find if players move is legal --in possible moves.
        if move not in pm: print(str(move)+" is not legal."); return 0
        
        #extract possition column
        game_column = [row[move] for row in self.game]
        
        for i in range(self.row-1,-1,-1):
            if game_column[i] == 0: self.game[i][move] = p1[1]; break
                
        #print new position
        os.system('cls')
        print(p1[0]+" tossed a piece in possition ("+str(i)+","+str(move)+")")
        self.print_position()
        return self.check_for_win(p1,p2)
    
    def check_for_win(self,p1=['NAME1','X'],p2=['AI','O']):
        
        #check win horizontaly (in each row)
        game0 = self.game
        for i in range(len(game0)):
            for j in range(len(game0[i])-4+1):
                if game0[i][j:j+4]==[p1[1]]*4: print("A horizantal win for "+p1[0]); return True
                if game0[i][j:j+4]==[p2[1]]*4: print("A horizantal win for "+p2[0]); return True
        
        #check win verticaly (in each column)
        game90 = [[row[i] for row in self.game] for i in range(self.column)]
        for i in range(len(game90)):
            for j in range(len(game90[i])-4+1):
                if game90[i][j:j+4]==[p1[1]]*4: print("A vertical win for "+p1[0]); return True
                if game90[i][j:j+4]==[p2[1]]*4: print("A vertical win for "+p2[0]); return True
        
        #check for diag-horizontal (\ \ \)
        game45 = [list(np.diag(np.asarray(game0),i)) for i in range(-len(game0)+1,len(game0[0]))]
        for i in range(len(game45)):
            for j in range(len(game45[i])-4+1):
                if game45[i][j:j+4]==[p1[1]]*4: print("A diagonal win for "+p1[0]); return True
                if game45[i][j:j+4]==[p2[1]]*4: print("A diagonal win for "+p2[0]); return True                
        
        #check for diag-vertical (/ / /)
        game135 = [list(np.diag(np.asarray(game90[::-1]),i)) for i in range(-len(game90)+1,len(game90[0]))]
        for i in range(len(game135)):
            for j in range(len(game135[i])-4+1):
                if game135[i][j:j+4]==[p1[1]]*4: print("A diagonal win for "+p1[0]); return True
                if game135[i][j:j+4]==[p2[1]]*4: print("A diagonal win for "+p2[0]); return True  
                
        return False
        
    def print_position(self):
        print("",end="  ")
        for j in range(self.column):
            print("   "+str(j),end="")
        print("")
        for i in range(self.row):
            print(i,end="  | ")
            for j in range(self.column):
                if self.game[i][j]!=0: print(self.game[i][j], end=" | ")
                else: print(" ", end=" | ")
            print("")
            

C:\Users\lenovo\AppData\Roaming\Python\Python38\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\lenovo\AppData\Roaming\Python\Python38\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\lenovo\AppData\Roaming\Python\Python38\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import copy

class MinMax_AI():
    row: int
    column: int
    max_dep: int
    AI: int
    Opn: int
        
    def __init__(self,max_dep,row,column,Opn=['PLAYER','X'],AI=['AI','O']):
        self.max_dep = max_dep
        self.row = row
        self.column = column
        self.AI = AI
        self.Opn = Opn
        
    def search(self,game):
        crr_dep = 0
        position = copy.deepcopy(game)
        best_choice = self.max_search(position,crr_dep)
        print(best_choice)
        return best_choice[1]
    
    def min_search(self,position,dep):
        pm = self.possible_moves(position)
        values = []
        
        if dep>=self.max_dep or self.check_for_win(position) or len(pm)==0: return [self.evaluate_(copy.deepcopy(position),self.Opn,self.AI),-1]
        
        for move in pm:
            values.append([self.max_search(copy.deepcopy(self.player_move(copy.deepcopy(position),move,['Blue','X'])),dep+1)[0],move])

        return min(values)
    
    
    def max_search(self,position,dep):
        pm = self.possible_moves(position)
        values = []
        
        if dep>=self.max_dep or self.check_for_win(position) or len(pm)==0: return [self.evaluate_(copy.deepcopy(position),self.Opn,self.AI),-1]
        
        for move in pm:
            values.append([self.min_search(copy.deepcopy(self.player_move(copy.deepcopy(position),move,['AI','O'])),dep+1)[0],move])

        return max(values)
        
            
        
    def evaluate(self,position,p1=['PLAYER','X'],p2=['AI','O']):
        
        #value of this position
        value = 0
        
        #check win horizontaly (in each row)
        game0 = copy.deepcopy(position)
        for i in range(len(game0)):
            for j in range(len(game0[i])-4+1):
                if game0[i][j:j+4]==[p1[1]]*4: value-=200; break
                if game0[i][j:j+4]==[p2[1]]*4: value+=100; break
            for j in range(len(game0[i])-3+1):
                if game0[i][j:j+3]==[p1[1]]*3: value-=100; break
                if game0[i][j:j+3]==[p2[1]]*3: value+=5; break
            for j in range(len(game0[i])-2+1):
                if game0[i][j:j+2]==[p1[1]]*2: value+=5; break
                if game0[i][j:j+2]==[p2[1]]*2: value-=2; break
        
        #check win verticaly (in each column)
        game90 = [[row[i] for row in position] for i in range(self.column)]
        for i in range(len(game90)):
            for j in range(len(game90[i])-4+1):
                if game90[i][j:j+4]==[p1[1]]*4: value-=200; break
                if game90[i][j:j+4]==[p2[1]]*4: value+=100; break
            for j in range(len(game90[i])-3+1):
                if game90[i][j:j+3]==[p1[1]]*3: value-=100; break
                if game90[i][j:j+3]==[p2[1]]*3: value+=5; break
            for j in range(len(game90[i])-2+1):
                if game90[i][j:j+2]==[p1[1]]*2: value-=5; break
                if game90[i][j:j+2]==[p2[1]]*2: value+=2; break
        
        #check for diag-horizontal (\ \ \)
        game45 = [list(np.diag(np.asarray(game0),i)) for i in range(-len(game0)+1,len(game0[0]))]
        for i in range(len(game45)):
            for j in range(len(game45[i])-4+1):
                if game45[i][j:j+4]==[p1[1]]*4: value-=200; break
                if game45[i][j:j+4]==[p2[1]]*4: value+=100; break
            for j in range(len(game45[i])-3+1):
                if game45[i][j:j+3]==[p1[1]]*3: value-=100; break
                if game45[i][j:j+3]==[p2[1]]*3: value+=5; break
            for j in range(len(game45[i])-2+1):
                if game45[i][j:j+2]==[p1[1]]*2: value-=5; break
                if game45[i][j:j+2]==[p2[1]]*2: value+=2; break 
        
        #check for diag-vertical (/ / /)
        game135 = [list(np.diag(np.asarray(game90[::-1]),i)) for i in range(-len(game90)+1,len(game90[0]))]
        for i in range(len(game135)):
            for j in range(len(game135[i])-4+1):
                if game135[i][j:j+4]==[p1[1]]*4: value-=200; break
                if game135[i][j:j+4]==[p2[1]]*4: value+=100; break
            for j in range(len(game135[i])-3+1):
                if game135[i][j:j+3]==[p1[1]]*3: value-=100; break
                if game135[i][j:j+3]==[p2[1]]*3: value+=5; break
            for j in range(len(game135[i])-2+1):
                if game135[i][j:j+2]==[p1[1]]*2: value-=5; break
                if game135[i][j:j+2]==[p2[1]]*2: value+=2; break
                
        return value

    
    def evaluate_(self,position,p1=['PLAYER','X'],p2=['AI','O']):
        
        #value of this position
        value = 0
        
        #check win horizontaly (in each row)
        game0 = copy.deepcopy(position)
        for i in range(len(game0)):
            for j in range(len(game0[i])-4+1):
                value -= self.add_cmp(game0[i][j:j+4],[p1[1]]*4)
                value += self.add_cmp(game0[i][j:j+4],[p2[1]]*4)

        
        #check win verticaly (in each column)
        game90 = [[row[i] for row in position] for i in range(self.column)]
        for i in range(len(game90)):
            for j in range(len(game90[i])-4+1):
                value -= self.add_cmp(game90[i][j:j+4],[p1[1]]*4)
                value += self.add_cmp(game90[i][j:j+4],[p2[1]]*4)

        
        #check for diag-horizontal (\ \ \)
        game45 = [list(np.diag(np.asarray(game0),i)) for i in range(-len(game0)+1,len(game0[0]))]
        for i in range(len(game45)):
            for j in range(len(game45[i])-4+1):
                value -= self.add_cmp(game45[i][j:j+4],[p1[1]]*4)
                value += self.add_cmp(game45[i][j:j+4],[p2[1]]*4)

        
        #check for diag-vertical (/ / /)
        game135 = [list(np.diag(np.asarray(game90[::-1]),i)) for i in range(-len(game90)+1,len(game90[0]))]
        for i in range(len(game135)):
            for j in range(len(game135[i])-4+1):
                value -= self.add_cmp(game135[i][j:j+4],[p1[1]]*4)
                value += self.add_cmp(game135[i][j:j+4],[p2[1]]*4)

                
        return value

    def evaluate__(self,position,p1=['PLAYER','X'],p2=['AI','O']):
        
        #value of this position
        value = 0
        
        #check win horizontaly (in each row)
        game0 = copy.deepcopy(position)
        for i in range(len(game0)):
            for j in range(len(game0[i])-4+1):
                value -= self.add_cmp(game0[i][j:j+4],[p1[1]]*4)*2
                value += self.add_cmp(game0[i][j:j+4],[p2[1]]*4)

        
        #check win verticaly (in each column)
        game90 = [[row[i] for row in position] for i in range(self.column)]
        for i in range(len(game90)):
            for j in range(len(game90[i])-4+1):
                value -= self.add_cmp(game90[i][j:j+4],[p1[1]]*4)*2
                value += self.add_cmp(game90[i][j:j+4],[p2[1]]*4)

        
        #check for diag-horizontal (\ \ \)
        game45 = [list(np.diag(np.asarray(game0),i)) for i in range(-len(game0)+1,len(game0[0]))]
        for i in range(len(game45)):
            for j in range(len(game45[i])-4+1):
                value -= self.add_cmp(game45[i][j:j+4],[p1[1]]*4)*2
                value += self.add_cmp(game45[i][j:j+4],[p2[1]]*4)

        
        #check for diag-vertical (/ / /)
        game135 = [list(np.diag(np.asarray(game90[::-1]),i)) for i in range(-len(game90)+1,len(game90[0]))]

        for i in range(len(game135)):
            for j in range(len(game135[i])-4+1):
                value -= self.add_cmp(game135[i][j:j+4],[p1[1]]*4)*2
                value += self.add_cmp(game135[i][j:j+4],[p2[1]]*4)

                
        return value

    
    def add_cmp(self,array1,array2):
        counter=0
        for i in range(len(array1)):
            if array1[i]!=0:
                counter+=1
                if array1[i]!=array2[i]:return 0
        if counter==4: return 1000
        return 1
    
    def possible_moves(self,position):
        #find all possible moves in a possition
        pm = list(range(self.column))
        pm_copy = pm[:]
        for i in pm_copy:
            if position[0][i]!=0: pm.remove(i)
        return pm
    
    def player_move(self,position,move,name=['PLAYER','X']):
        #extract possition column
        game_column = [row[move] for row in position]

        for i in range(self.row-1,-1,-1):
            if game_column[i] == 0: position[i][move] = name[1]; break
                
        return position
    
    def check_for_win(self,position,p1=['PLAYER','X'],p2=['AI','O']):
        
        #check win horizontaly (in each row)
        game0 = position
        for i in range(len(game0)):
            for j in range(len(game0[i])-4+1):
                if game0[i][j:j+4]==[p1[1]]*4: return True
                if game0[i][j:j+4]==[p2[1]]*4: return True
        
        #check win verticaly (in each column)
        game90 = [[row[i] for row in position] for i in range(self.column)]
        for i in range(len(game90)):
            for j in range(len(game90[i])-4+1):
                if game90[i][j:j+4]==[p1[1]]*4: return True
                if game90[i][j:j+4]==[p2[1]]*4: return True
        
        #check for diag-horizontal (\ \ \)
        game45 = [list(np.diag(np.asarray(game0),i)) for i in range(-len(game0)+1,len(game0[0]))]
        for i in range(len(game45)):
            for j in range(len(game45[i])-4+1):
                if game45[i][j:j+4]==[p1[1]]*4: return True
                if game45[i][j:j+4]==[p2[1]]*4: return True                
        
        #check for diag-vertical (/ / /)
        game135 = [list(np.diag(np.asarray(game90[::-1]),i)) for i in range(-len(game90)+1,len(game90[0]))]
        for i in range(len(game135)):
            for j in range(len(game135[i])-4+1):
                if game135[i][j:j+4]==[p1[1]]*4: return True
                if game135[i][j:j+4]==[p2[1]]*4: return True  
                
        return False

In [3]:
class AlphaBeta_AI(MinMax_AI):
    
    def search(self,game):
        crr_dep = 0
        position = copy.deepcopy(game)
        best_choice = self.max_search(position,crr_dep,-2000000,+2000000)
        print(best_choice)
        return best_choice[1]
        
        
    def max_search(self,position,dep,a,b):
        pm = self.possible_moves(position)
        v = [-2000000,-1]
        if dep>=self.max_dep or self.check_for_win(position) or len(pm)==0: return [self.evaluate_(copy.deepcopy(position),self.Opn,self.AI),-1]

        for move in pm:
            #change to self.ai
            v_n = self.min_search(copy.deepcopy(self.player_move(copy.deepcopy(position),move,['AI','O'])),dep+1,a,b)[0]
            if v_n>=v[0]: v=[v_n,move]
            if v[0]>b: return v
            a = max(a,v[0])
    
        return v
    
            
    def min_search(self,position,dep,a,b):
        pm = self.possible_moves(position)
        v = [+2000000,-1]
        if dep>=self.max_dep or self.check_for_win(position) or len(pm)==0: return [self.evaluate_(copy.deepcopy(position),self.Opn,self.AI),-1]

        for move in pm:
            v_n = self.max_search(copy.deepcopy(self.player_move(copy.deepcopy(position),move,['Blue','X'])),dep+1,a,b)[0]
            if v_n<=v[0]: v=[v_n,move]
            if v[0]<a: return v
            b = min(b,v[0])

        return v


Define A* agent in a way that is presented in the report

In [4]:
class Astar_AI(MinMax_AI):
    max_queue = []
    min_queue = []
    def search(self,game):
        crr_dep = 0
        self.max_queue = []
        self.min_queue = []
        position = copy.deepcopy(game)
        pm = self.possible_moves(position)
        #print(position)
        if self.check_for_win(position) or len(pm)==0: return -1
        
        #self.max_queue.append([self.evaluate_(position),copy.deepcopy(position),crr_dep,[-1]])
        
        for move in pm:
            pi = copy.deepcopy(self.player_move(copy.deepcopy(position),move,self.AI))
            self.max_queue.append([self.evaluate_(pi),copy.deepcopy(pi),crr_dep,[-1]])
        self.max_queue.sort(reverse=True, key=lambda x: x[0])
        best_choice = self.loop_search()
        #print(best_choice)
        if len(best_choice[3])>1: return best_choice[3][1]
        return -1
    
    def max_extend(self):
        if len(self.max_queue)==0: return [-1]
        e_node = self.max_queue[0]
        self.max_queue = self.max_queue[1:]
        
        #determine possible moves
        p_moves = self.possible_moves(e_node[1])
        
        #check being goal
        print(len(self.max_queue))
        if self.check_for_win(e_node[1]) or e_node[2]>self.max_dep: return [1,e_node]
        if len(p_moves)==0: pass
        
        #extend node and add to queue
        for move in p_moves:
            pi = copy.deepcopy(self.player_move(copy.deepcopy(e_node[1]),move,self.Opn))
            self.min_queue.append([self.evaluate_(pi),copy.deepcopy(pi),e_node[2]+1,e_node[3][:]+[move]])
        
        #exploit min nodes
        self.min_queue.sort(key=lambda x: x[0])
        for i in range(min(2,len(self.min_queue))):
            self.min_extend()
            
        self.min_queue = []
        return [-1]
        
    def min_extend(self):
        e_node = self.min_queue[0]
        self.min_queue = self.min_queue[1:]
        #print("min extend: ", e_node)
        #determine possible moves
        p_moves = self.possible_moves(e_node[1])
        
        #check being goal
        if self.check_for_win(e_node[1]) or e_node[2]>self.max_dep: return [1,e_node]
        if len(p_moves)==0: pass
        #extend node and add to queue
        for move in p_moves:
            pi = copy.deepcopy(self.player_move(copy.deepcopy(e_node[1]),move,self.AI))
            self.max_queue.append([self.evaluate_(pi),copy.deepcopy(pi),e_node[2]+1,e_node[3][:]+[move]])
        
        self.max_queue.sort(reverse=True, key=lambda x: x[0])
        
    def loop_search(self,d=int(2e10)):
        for i in range(d):
            t = self.max_extend()
            #print(t)
            if t[0]==1: return t[1]
            if t[0]==0: return False
        if (self.max_queue)>=1:return self.max_queue[0]
        return [-1,-1,-1,[-1,self.a_possible_move()]]
    
    def a_possible_move(self):
        position = copy.deepcopy(game)
        pm = self.possible_moves(position)
        return pm[0]

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

class FCN_AI(nn.Module):
    def __init__(self):
        super(FCN_AI, self).__init__()
        
        self.fc1 = nn.Linear(42, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 7)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        
        return x
    
#make a dataset to train by having AlphaBeta play with random player for 100 games
input_ = []
label_ = []
for game in range(100):
    r = cnctf(7,6,1)
    input_+=(r.input_)
    label_+=(r.label_)
    
#determine inputs and labels for training as np
inputs = torch.from_numpy(np.array([[ {0:0,'X':1,'O':2}.get(input_[j][i]) for i in range(42)] for j in range(len(input_))]).astype(np.float32))
labels = torch.from_numpy(np.array(label_))

#change the datatype
inputs = inputs.to(torch.float)
labels = labels.to(torch.long)


#instantiate the network
model = FCN_AI()

#define the loss function and the optimizer for classification
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

#number of training epochs
num_epochs = 10000

#train the network
for epoch in range(num_epochs):
    #forward pass
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    #backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    #print training loss after every 250 epochs
    if (epoch+1) % 1000 == 0:
        print(f"Epoch: {epoch+1}/{num_epochs}, Loss: {loss.item()}")

In [ ]:
#test it on a position
test = [[0,0,0,0,0,0,0,
        0,0,0,0,0,0,0,
        0,0,0,0,0,0,0,
        0,0,0,0,0,0,0,
        0,0,0,0,0,0,0,
        0,2,1,0,1,0,2]]

test = torch.from_numpy(np.array(test)).to(torch.float)
with torch.no_grad():
    output = model(test)
_,predict = torch.max(output.data, 1)  
print(predict)

##### A simple gameplay

In [6]:
r = cnctf(7,8)

**Welcome to Connect Four!**
Choose the game board size: 
Number of rows: 5
Number of columns: 6
What is the name of the player 1: Pouya
What is the player 1 piece (preferably X): X
Type AI or PLAYER regarding to player 1 agent: PLAYER
What is the name of the player 2: Agent
What is the player 2 piece (preferably O): O
Type AI or PLAYER regarding to player 2 agent: AI
Determine player 2 difficulty (2-5): 4
Choose a search alogorithm for player 2 (MinMax - AlphaBeta - A*): MinMax
Tosse a piece(0-5): 3
Pouya tossed a piece in possition (4,3)
     0   1   2   3   4   5
0  |   |   |   |   |   |   | 
1  |   |   |   |   |   |   | 
2  |   |   |   |   |   |   | 
3  |   |   |   |   |   |   | 
4  |   |   |   | X |   |   | 
[-3, 3]
Agent tossed a piece in possition (3,3)
     0   1   2   3   4   5
0  |   |   |   |   |   |   | 
1  |   |   |   |   |   |   | 
2  |   |   |   |   |   |   | 
3  |   |   |   | O |   |   | 
4  |   |   |   | X |   |   | 
Tosse a piece(0-5): 2
Pouya tossed a piece in possit